## Clustering challenge

Clustering is an unsupervised machine learning technique, in which you train a model to group similar entities into clusters based on their features.

In this exercise, you must separate a dataset consisting of three numeric features (*A*, *B*, and *C*) into clusters.

Let's begin by importing the data.


In [ ]:
# Load the core tidyverse and make it available in your current R session
suppressPackageStartupMessages(
  library(tidyverse))

# Initialize github repo path 
# containing test files used to check your answers
testsFolderPath <- "https://raw.githubusercontent.com/MicrosoftDocs/mslearn-machine-learning-with-r/main/tests/introduction-clustering-models/"

# Read the csv file into a tibble
clust_data <- read_csv(file = "https://raw.githubusercontent.com/MicrosoftDocs/ml-basics/master/challenges/data/clusters.csv", show_col_types = FALSE)

# Print the first 10 rows of the data
clust_data %>% 
  slice_head(n = 10)


Your challenge is to identify the number of discrete clusters present in the data, and create a clustering model that separates the data into that number of clusters. You should also visualize the clusters to evaluate the level of separation achieved by your model.

Add markdown and code cells as required to create your solution.

### Use PCA to create a 2D version of the features for visualization

*Principal component analysis* (PCA) is a dimension reduction method that attempts to reduce the feature space. With PCA, most of the information or variability in the dataset can be explained by using fewer uncorrelated features. Let's see this in action by creating a specification of a `recipe` that estimates the principal components, based on our three variables. You'll later `prep` and `bake` the recipe to apply the computations.


**Question 1**

For this question, create a specification of a `recipe` that will:
- Normalize all predictors.
- Convert all predictors into *two* principal components.

Fill in the placeholder `....` with the right code.


In [ ]:
set.seed(2056)
# Load the core tidymodels packages and make them available in your current R session
suppressPackageStartupMessages(
  library(tidymodels))

# Specify a recipe
pca_rec <- recipe(...., data = clust_data) %>%
  # Recipe step for normalizing predictors
  ....(all_predictors()) %>% 
  # Recipe step for converting all predictors into 2 Principal Components
  ....(...., num_comp = ...., id = "pca")

# Print out recipe
pca_rec


Test your answer:



In [ ]:
testFilePath <- paste(testsFolderPath, "Question%201.R", sep="", collapse=NULL)
. <- ottr::check(testFilePath)

Well done! Compared to supervised learning techniques, this technique has no outcome variable in this recipe.

Now, time to prep and bake the recipe!

**Question 2**

In this section:
- Prep the recipe you created above. Prepping a recipe estimates the required quantities and statistics required by PCA.
- Bake the prepped recipe. This takes the trained (prepped) recipe, and applies its operations to a dataset to create a design matrix. 

Fill in the placeholder `....` with the right code.

In [ ]:
# Estimate the recipe 
pca_estimates <- prep(....)

# Return preprocessed clust_data using bake
features_2d <- pca_estimates %>% 
  ....(new_data = NULL)

# Print baked data set
features_2d %>% 
  slice_head(n = 5)


Test your answer:

In [ ]:
testFilePath <- paste(testsFolderPath, "Question%202.R", sep="", collapse=NULL)
. <- ottr::check(testFilePath)

Good job! After prep and bake, you obtain two components that capture the maximum amount of information (that is, variance) in the original variables.

From the output of your prepped recipe `pca_estimates`, you can examine how much variance each component accounts for:


In [ ]:
# Examine how much variance each PC accounts for
pca_estimates %>% 
  tidy(id = "pca", type = "variance") %>% 
  filter(str_detect(terms, "percent"))


theme_set(theme_light())
# Plot how much variance each PC accounts for
pca_estimates %>% 
  tidy(id = "pca", type = "variance") %>% 
  filter(terms == "percent variance") %>% 
  ggplot(mapping = aes(x = component, y = value)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("% of total variance")


This output's tibbles and plots show how well each principal component is explaining the original six variables. For example, the first principal component (PC1) explains about 84.93 percent of the variance of the three variables. The second principal component explains an additional 7.63 percent, giving a cumulative percent variance of 92.56 percent. This is certainly better. It means that the first two variables seem to have some power in summarizing the original three variables.

Now that you have the data points translated to two dimensions PC1 and PC2, you can visualize them in a plot:


In [ ]:
# Visualize PC scores
features_2d %>% 
  ggplot(mapping = aes(x = PC1, y = PC2)) +
  geom_point(size = 2, color = "dodgerblue3")


### Calculate WCSS for multiple cluster numbers to determine the right number of clusters

Here's one of the fundamental problems with clustering: without knowing class labels, how do you know how many clusters to separate your data into?


**Question 3**

In this section, you'll try to find the optimal number of clusters that your data can be divided into. You do this by using your data sample, `clust_data`, to create a series of clustering models with an incrementing number of clusters. Then you measure how tightly the data points are grouped within each cluster.

A metric often used to measure this tightness is the *within cluster sum of squares* (WCSS), with lower values meaning that the data points are closer. You can then plot the WCSS for each model.

- Prep and bake `clust_data` to obtain a new tibble, where all the predictors are normalized.
- Use the built-in `kmeans()` function to perform clustering, by creating a series of 10 clustering models.
- Extract the WCSS, `tot.withinss`, for each cluster.

Fill in the placeholder `....` with the right code.


In [ ]:
# Obtain normalized data for estimating clusters
clust_estimate_data <- recipe(~ ., data = clust_data) %>%
  # Step to normalize all predictors
  ....(all_predictors()) %>% 
  # Prep the recipe
  .... %>% 
  # Apply trained recipe to train set: clust_data
  ....

# Print out data
clust_estimate_data %>% 
  slice_head(n = 5)


Now, let's explore the WCSS of different numbers of clusters.

You can use `map()` from the [purrr](https://purrr.tidyverse.org/) package to apply functions to each element in list.

> [`map()`](https://purrr.tidyverse.org/reference/map.html) functions allow you to replace many `for` loops with code that is both more succinct and easier to read. The best place to learn about the `map()` functions is the [iteration chapter](http://r4ds.had.co.nz/iteration.html) in R for data science.
>
> The function `broom::glance.kmeans()` accepts a model object, and returns a tibble with exactly one row of model summaries. The summaries are typically goodness of fit measures, p-values for hypothesis tests on residuals, or model convergence information.


In [ ]:
set.seed(2056)
# Create 10 models with 1 to 10 clusters
kclusts <- tibble(k = ....) %>% 
  mutate(
    # Create a series of clustering models with centers k
    model = map(k, ~ kmeans(x = clust_estimate_data, centers = .x, nstart = 20)),
    # Extract the Total WCSS tot.withinss, for each model
    tot.withinss = map_dbl(...., ~ glance(.x)$tot.withinss))



# Plot Total within-cluster sum of squares (tot.withinss) for each model with center k
kclusts %>% 
  ggplot(mapping = aes(x = k, y = ....)) +
  geom_line(size = 1.2, alpha = 0.5, color = "dodgerblue3") +
  geom_point(size = 2, color = "dodgerblue3")


Test your answer:

In [ ]:
testFilePath <- paste(testsFolderPath, "Question%203.R", sep="", collapse=NULL)
. <- ottr::check(testFilePath)

That went well! What do you think the chart is telling you?



You want to minimize the total within-cluster sum of squares, by performing K-means clustering. The plot shows a large reduction in WCSS (so, greater tightness) as the number of clusters increases from one to two. The plot shows a further noticeable reduction from two to three, and then three to four clusters. After that, the reduction is less pronounced, resulting in an elbow in the chart at around four clusters. This is a good indication that there are four reasonably well-separated clusters of data points.

### Use K-means

**Question 4**


In the previous section, you noticed that there are four reasonably well-separated clusters of data points. In this section:

Extract the model that separates the data into k = 4 clusters. Starting with the clustering results `kclusts`:
1. Create a subset where the models contains `k = 4` clusters.
1. Use `pull()` to extract the clustering model in the `model` column.
1. Retrieve the model components, such as cluster assignments, by using `pluck()`.

Fill in the placeholder `....` with the right code.

In [ ]:
# Extract the model with k = 4 clusters
final_kmeans <- kclusts %>% 
  # Narrow down to the model with K = 4 centers
  filter(....) %>%
  # Extract the clustering model
  ....(model) %>% 
  # Retrieve model components
  ....(1)

# Print model components
final_kmeans


Test your answer:

In [ ]:
testFilePath <- paste(testsFolderPath, "Question%204.R", sep="", collapse=NULL)
. <- ottr::check(testFilePath)

What can you make out of the K-means object, `final_kmeans`? One of the available components is the *clustering vector*: a vector of integers (from 1 to k) indicating the cluster to which each observation in your data is allocated. Be sure to explore the other available components.


### Plot the clustered points

Great! Let's go ahead and visualize the clusters obtained.

**Question 5**

In this section:

- Add the cluster assignments generated by the model to the 2D version (2 principal components) of your dataset. That is, you're adding the assignments to `features_2d`. Hint: `augment` accepts a model object and a dataset, and adds information about each observation in the dataset.

- Visualize the cluster assignments by using a scatter plot. Assign the `shape` and `color` aesthetics to the cluster number assigned to each data point.

Fill in the placeholder `....` with the right code.

In [ ]:
# Load plotting libraries 
suppressPackageStartupMessages({
  library(plotly)
  library(paletteer)
})


# Augment featured_2d with information from the clustering model
results <-  augment(...., ....) 

# Print some rows of the results
results %>% 
  slice_head(n = 10)


Voila! There go the two principal components (PC1 and PC2), which capture the maximum amount of information in the original variables, along with their corresponding cluster assignment (*.cluster*).

Now, let's see this in an interactive plot!


In [ ]:
# Plot cluster assignments
cluster_plot <- results %>% 
  ggplot(mapping = aes(x = PC1, y = PC2)) +
  geom_point(aes(...., ....), size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggsci::category10_d3")

# Make plot interactive
ggplotly(cluster_plot)


Test your answer:

In [ ]:
testFilePath <- paste(testsFolderPath, "Question%205.R", sep="", collapse=NULL)
. <- ottr::check(testFilePath)

Hopefully you're able to observe that the data has been separated into four distinct clusters. 

K-means is a commonly used clustering algorithm that separates a dataset into K clusters of equal variance. Observations within the same cluster are as similar as possible (that is, they have high intra-class similarity), whereas observations from different clusters are as dissimilar as possible (they have low inter-class similarity).

The first step in K-means clustering is the data scientist specifying the number of clusters K to partition the observations into. Hierarchical clustering is an alternative approach, which doesn't require the number of clusters to be defined in advance. Let's jump right into it.

### Try agglomerative clustering

Hierarchical clustering creates clusters by either a *divisive* method or an *agglomerative* method. The divisive method is a top-down approach, starting with the entire dataset, and then finding partitions in a stepwise manner. Agglomerative clustering is a bottom-up approach, in which each observation is initially considered as a single element cluster. At each step of the algorithm, two clusters that are most similar are combined into a bigger cluster, until all points are members of one single, big cluster.


**Question 6**

In this section, you'll cluster the data by using an agglomerative clustering algorithm. There are many functions available in R for hierarchical clustering.

The `hclust()` function is one way to perform hierarchical clustering. It requires a distance matrix structure, and a specification of the agglomeration method to be used. 

- Use the `dist` function to compute the distance matrix (distances between the rows of the data), by using the Euclidean method.

- Use `hclust` to perform hierarchical clustering with the `ward.D2` (Ward linkage) agglomeration method.

Fill in the placeholder `....` with the right code.


In [ ]:
# For reproducibility
set.seed(2056)

# Distance between observations matrix
d <- ....(x = clust_estimate_data, method = "....")

# Hierarchical clustering using Ward Linkage
hclust_ward <- ....(d, method = "....")


Test your answer:

In [ ]:
testFilePath <- paste(testsFolderPath, "Question%206.R", sep="", collapse=NULL)
. <- ottr::check(testFilePath)

The [factoextra](https://rpkgs.datanovia.com/factoextra/index.html) provides functions ([`fviz_dend()`](https://rdrr.io/pkg/factoextra/man/fviz_dend.html)) to visualize hierarchical clustering. Let's visualize the dendrogram representation of the clusters from the Ward linkage method.



In [ ]:
library(factoextra)

# Visualize cluster separations
fviz_dend(hclust_ward, main = "Ward Linkage")


Although hierarchical clustering doesn't require you to pre-specify the number of clusters, you still need to specify the number of clusters to extract.

You found out, by using the `WCSS` method, that the optimal number of clusters was 4. Let's color our dendrogram according to k = 4, and observe how observations will be grouped.


In [ ]:
# Visualize clustering structure for 4 groups
fviz_dend(hclust_ward, k = 4, main = "Ward Linkage")


Perfect! You can now go ahead and cut the hierarchical clustering model into four clusters, and extract the cluster labels for each observation associated with a particular cut. This is done by using `cutree()`.

**Question 7**

In this section, you cut the hierarchical clustering model into four clusters, and extract the cluster labels for each observation.

Starting with the principal components version of the data:

- Using the `cutree` function, extract k = 4 groups of data obtained from the hierarchical clustering process. Assign this to a column name `cluster_id`.

- Also, encode the `cluster_id` column as a factor.

Fill in the placeholder `....` with the right code.

In [ ]:
set.seed(2056)
# Extract cluster labels obtained from the Hierarchical clustering process
results_hclust <- features_2d %>% 
  ....(cluster_id = ....)


# Print some rows of the results
results_hclust %>% 
  slice_head(n = 5)


Test your answer:

In [ ]:
testFilePath <- paste(testsFolderPath, "Question%207.R", sep="", collapse=NULL)
. <- ottr::check(testFilePath)

Awesome! Now, let's inspect the cluster assignments visually.




### View the agglomerative cluster assignments


In [ ]:
# Plot h-cluster assignment on the PC data
hclust_spc_plot <- results_hclust %>% 
  ggplot(mapping = aes(x = PC1, y = PC2)) +
  geom_point(aes(shape = cluster_id, color = cluster_id), size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggsci::category10_d3")

# Make plot interactive
ggplotly(hclust_spc_plot)


How does this compare to the K-means results you visualized earlier?

You can see that the observations were grouped quite similarly by the two clustering algorithms, K-means and hierarchical. 

We'll wrap it at that! In this challenge, you created unsupervised machine learning models, to help you group unlabeled data observations into clusters. 

Congratulations on this amazing feat!

Happy Learning,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.
